In [2]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [3]:
#Elecion data
candidate_demo_df = pd.read_csv('../../cc_election_cleaning/electoral_clustering_with_pca.csv')
all_candidate_demo_df = pd.read_csv('../../cc_election_cleaning/all_candidate_electoral_clusters.csv')
weighted_demographics = pd.read_csv('../../cc_election_cleaning/weighted_demographics_clusters.csv')
#Finance data
winning_finance_cluster = pd.read_csv('../../council_campaign_finance/data/winning_members_clusters.csv')
all_finance_cluster = pd.read_csv('../../council_campaign_finance/data/all_members_clusters.csv')
#Sponsorship data
sponsorship_df = pd.read_csv('../../cc_legislation/data/cosponsorship_matrix_clusters.csv')

In [4]:
dldc = pd.read_csv("../../cc_election_cleaning/district_level_demo_clusters.csv")

In [5]:
dldc

,district,candidate,demo_cluster
0,1,Christopher Marte,3
1,2,Carlina Rivera,3
2,3,Erik D. Bottcher,3
3,4,Keith Powers,3
4,5,Julie Menin,3
5,6,Gale A. Brewer,3
6,7,Shaun Abreu,0
7,8,Diana I. Ayala,0
8,9,Kristin Richardson Jordan,0
9,10,Carmen N. De La Rosa,0


In [90]:
def match_names(name, names_list):
    match, score = process.extractOne(name, names_list, scorer=fuzz.token_sort_ratio)
    return match if score >= 90 else name

In [91]:
# clusters and meta cluster from voting and sponsorship
meta_clusters = pd.read_csv('../../cc_legislation/data/meta_clusters.csv')

In [92]:
faction_compare = pd.read_csv("../../citycouncil_tweets/data/faction_compare.csv")

In [93]:
name_list = faction_compare['VotePersonName'].tolist()

In [94]:
meta_clusters['proper_name'] = meta_clusters['member'].apply(lambda x: match_names(x, name_list))

Add other clusters

In [95]:
finance = pd.read_csv('../../council_campaign_finance/data/winning_members_clusters_with_gop.csv')

In [96]:
finance = finance[['normalized_mem', 'cluster']]

In [97]:
finance.columns = ['normalized_name', 'finance_cluster']

In [98]:
finance['proper_name'] = finance['normalized_name'].apply(lambda x: match_names(x, name_list))

In [99]:
dldc['candidate'] = dldc['candidate'].astype(str)

In [100]:
dldc['proper_name'] = dldc['candidate'].apply(lambda x: match_names(x, name_list))

In [103]:
# change value for proper name in index 31
finance.at[31, 'proper_name'] = 'Kristin Richardson Jordan'

In [104]:
finance.at[8, 'proper_name'] = 'Chi A. Ossé'

In [105]:
dldc.at[47, 'proper_name'] = 'Inna Vernikov'

In [106]:
meta_clusters = pd.merge(meta_clusters, finance, on='proper_name', how='left')

In [27]:
# if normalized_mem is null, then assign value from proper_name
meta_clusters['normalized_mem'] = meta_clusters['normalized_mem'].fillna(meta_clusters['proper_name'])

In [111]:
left_pc = ['Erik Bottcher', 'Keith Powers', 'Julie Menin','Gale Brewer', 'Shaun Abreu', 'Diana I Ayala', 'Eric Dinowitz', 'Kevin C Riley', 'Marjorie Velazquez', 'Lynn Schulman', 'Selvena Brooks-Powers', 'Darlene Mealy', 'Justin Brannan', 'Farah Louis', 'Mercedes Narcisse']
stayed = ['Christopher Marte', 'Carlina Rivera', 'Carmen De La Rosa', 'Pierina A Sanchez', 'Althea V Stevens', 'Amanda Farias', 'Tiffany Caban', 'Shekar Krishnan', 'Julie Won', 'Nantasha Williams', 'Adrienne Adams', 'Lincoln Restler', 'Jennifer Gutierrez', 'Crystal Hudson', 'Chi Osse', 'Sandy Nurse', 'Alexa Aviles', 'Shahana Hanif', 'Rita C Joseph', 'Charles Barron','Kristin R Jordan']

In [112]:
# if normalized_mem is in in left_pc, then assign 'pc' a value of 1, if normalized_mem is in stayed, then assign 'pc' a value of 2, else assign 'pc' a value of 0
meta_clusters['pc'] = np.where(meta_clusters['normalized_name'].isin(left_pc), 1, np.where(meta_clusters['normalized_name'].isin(stayed), 2, 0))

In [114]:
meta_clusters = pd.merge(meta_clusters, dldc[['proper_name','demo_cluster','district']], left_on='proper_name', right_on='proper_name', how='left')

In [115]:
meta_clusters.at[6, 'district'] = 43
meta_clusters.at[33, 'district'] = 4
meta_clusters.at[47, 'district'] = 51
meta_clusters.at[49, 'district'] = 44

In [116]:
meta_clusters

,member,k_spon,louv,vote_cluster,kmode_cluster,k_meta_cluster,proper_name,normalized_name,finance_cluster,pc,demo_cluster,district
0,rita c. joseph,0,0,0,0,1,Rita C. Joseph,Rita C Joseph,3,2,4.0,40.0
1,julie menin,0,1,0,2,0,Julie Menin,Julie Menin,0,1,3.0,5.0
2,kristin richardson jordan,0,0,2,3,4,Kristin Richardson Jordan,Kristin R Jordan,3,2,0.0,9.0
3,sandra ung,0,2,0,2,2,Sandra Ung,Sandra Ung,0,0,2.0,20.0
4,eric dinowitz,0,1,0,2,0,Eric Dinowitz,Eric Dinowitz,4,1,0.0,11.0
5,diana i. ayala,0,2,0,2,2,Diana I. Ayala,Diana I Ayala,2,1,0.0,8.0
6,justin l. brannan,0,1,0,2,0,Justin L. Brannan,Justin Brannan,4,1,1.0,43.0
7,selvena n. brooks-powers,0,1,0,2,0,Selvena N. Brooks-Powers,Selvena Brooks-Powers,2,1,4.0,31.0
8,christopher marte,0,0,0,0,1,Christopher Marte,Christopher Marte,0,2,3.0,1.0
9,nantasha m. williams,0,2,0,2,2,Nantasha M. Williams,Nantasha Williams,3,2,4.0,27.0


In [26]:
meta_clusters.to_csv('../data/meta_clusters_with_formatted_name.csv', index=False)